In [ ]:
import ee
import geemap
import geopandas as gpd

ee.Authenticate()

ee.Initialize(project='ee-mhfulan-ic')

In [ ]:
import os
import sys

print()

In [ ]:
# Load arapressao shapefile as GeoDataFrame
arapressao = gpd.read_file(f"{(os.path.abspath("")).strip('python')}\example-data/arapressao-limits/arapressao.shp")

# GeoDtaFrame to ee.FeatureCollection
arapressao = geemap.geopandas_to_ee(arapressao)

In [ ]:
start_year = 1985
end_year = 2022

scale_chirps = 5000
sample_num_pixels = 300
sample_num_clusters = 8

# LOADS STUDY AREA SHAPEFILE AS 'arapressao'
study_area_feature_collection = ee.FeatureCollection(arapressao)
study_area = study_area_feature_collection.geometry()

# ----- CHIRPS -----
collection_chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").select("precipitation")

accumulated_chirps = collection_chirps \
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
    .sum() \
    .clip(study_area) \
    .set('year', end_year) \
    .set('system:time_start', ee.Date.fromYMD(start_year, 1, 1))

def apply_kmeans(image, num_clusters):
    training = image.sample(
        region=study_area,
        scale=scale_chirps,
        numPixels=sample_num_pixels
    )
    
    clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(training)
    result = image.cluster(clusterer)
    
    return result.set('year', image.get('year')) \
        .set('system:time_start', image.get('system:time_start'))

clustered_chirps = apply_kmeans(accumulated_chirps, sample_num_clusters)

def add_year_band(image, year):
    return image.addBands(ee.Image.constant(year).rename('year')).float()

def create_year_image(year):
    year = ee.Number(year)
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year.add(1), 1, 1)

    annualImage = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD") \
                  .filterDate(startDate, endDate) \
                  .select('precipitation') \
                  .sum()

    return add_year_band(annualImage, year).set('year', year)

annual_collection = ee.List.sequence(start_year, end_year).map(create_year_image)
annual_collection = ee.ImageCollection(annual_collection)

sens_slope = annual_collection.select(['year', 'precipitation']).reduce(ee.Reducer.sensSlope()).clip(study_area)

clean_sens_slope = sens_slope.select('slope').clip(study_area)

sens_slope_clustered = apply_kmeans(clean_sens_slope, sample_num_clusters)

# Corrected focalMean call
smoothed_sens_slope_clustered = apply_kmeans(
    clean_sens_slope.focalMean(radius=15000, units='meters').clip(study_area), 
    sample_num_clusters
)

In [ ]:
Map = geemap.Map(center = [-21, -50.5], zoom = 7)
Map.addLayer(clustered_chirps, {'min':0, 'max':7, 'palette':['#fde725', '#a0da39', '#4ac16d', '#1fa187', '#277f8e', '#365c8d', '#46327e', '#440154']}, 'KMeans CHIRPS')
Map.addLayer(sens_slope_clustered, {'min':0, 'max':7, 'palette':['#fde725', '#a0da39', '#4ac16d', '#1fa187', '#277f8e', '#365c8d', '#46327e', '#440154']}, "Sen's slope CHIRPS")
Map.addLayer(smoothed_sens_slope_clustered, {'min':0, 'max':7, 'palette':['#fde725', '#a0da39', '#4ac16d', '#1fa187', '#277f8e', '#365c8d', '#46327e', '#440154']}, "Smoothed Sen's slope CHIRPS")
Map